# Compare to timm-pretrained model, then turn into series of unit tests

In [1]:
from vit_prisma.configs import HookedViTConfig
from vit_prisma.models.base_vit import HookedViT

import timm
import torch

In [2]:
prisma_model = HookedViT.from_pretrained("vit_base_patch32_224", 
#                                          center_writing_weights=False, 
#                                          fold_ln=False, 
#                                          fold_value_biases=False,
#                                          use_attn_scale=False,
#                                          use_attn_in=True,
)
# timm_model = timm.create_model('vit_base_patch16_224', pretrained=True)

hf config ViTConfig {
  "_name_or_path": "timm/vit_base_patch32_224.augreg_in21k_ft_in1k",
  "architecture": "vit_base_patch32_224",
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "global_pool": "token",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_classes": 1000,
  "num_features": 768,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pretrained_cfg": {
    "classifier": "head",
    "crop_mode": "center",
    "crop_pct": 0.9,
    "custom_load": true,
    "first_conv": "patch_embed.proj",
    "fixed_input_size": true,
    "input_size": [
      3,
      224,
      224
    ],
    "interpolation": "bicubic",
    "mean": [
      0.5,
      0.5,
      0.5
    ],
    "num_classes": 1000,
    "pool_size": null,
    "std": [
      0.5,
      0.5,
      0.5
    ],

RuntimeError: Error(s) in loading state_dict for HookedViT:
	size mismatch for embed.proj.weight: copying a param with shape torch.Size([768, 3, 32, 32]) from checkpoint, the shape in current model is torch.Size([768, 3, 16, 16]).
	size mismatch for pos_embed.W_pos: copying a param with shape torch.Size([50, 768]) from checkpoint, the shape in current model is torch.Size([197, 768]).

In [ ]:
timm_model = timm.create_model('vit_base_patch16_224', pretrained=True)

In [ ]:
timm_model

In [ ]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalize the images to [-1, 1]
    # Resize to 224 x 224
    transforms.Resize((224, 224))
])

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)


In [ ]:
image, label = next(iter(testloader))
output, cache = prisma_model.run_with_cache(image)

for key in cache.keys():
    print(key, cache[key].shape)

## Patch Embeddings

In [ ]:
activations = []
def hook_fn(module, input, output):
    activations.append(output)

hook_handle = timm_model.patch_embed.register_forward_hook(hook_fn)
timm_output = timm_model(image)
hook_handle.remove()

activations[0].shape

assert torch.allclose(activations[0], cache['embed'][0])
assert torch.all(activations[0] == cache['embed'][0])

## Position Embeddings

In [ ]:
# First layer
activations = []
def hook_fn(module, input, output):
    activations.append(output)

hook_handle = timm_model.pos_drop.register_forward_hook(hook_fn)
timm_output = timm_model(image)
hook_handle.remove()

assert torch.allclose(activations[0], cache['blocks.0.hook_resid_pre'], atol=1e-6), "Activations differ more than the allowed tolerance"

## Layers

### LayerNorm 1

In [ ]:

import einops 

activations = []
def hook_fn(module, input, output):
    activations.append(output)

hook_handle = timm_model.blocks[0].norm1.register_forward_hook(hook_fn)
timm_output = timm_model(image)
hook_handle.remove()

print(activations[0].shape)
print(cache['blocks.0.ln1.hook_normalized'].shape)

# Assert equal to the first layer
assert torch.allclose(activations[0], cache['blocks.0.ln1.hook_normalized'][0], atol=1e-6), "Activations differ more than the allowed tolerance"

### Attention

**Weights**

In [ ]:
# # let's compare qkv weights
# QKV = timm_model.blocks[0].attn.qkv.weight
# W_Q, W_K, W_V = torch.tensor_split(QKV, 3, dim=0)
# t_Q = einops.rearrange(W_Q, "(i h) m->h m i", h=12)
# t_K = einops.rearrange(W_K, "(i h) m->h m i", h=12)
# t_V = einops.rearrange(W_V, "(i h) m->h m i", h=12)

# p_Q = prisma_model.blocks[0].attn.W_Q
# p_K = prisma_model.blocks[0].attn.W_K
# p_V = prisma_model.blocks[0].attn.W_V

# assert torch.allclose(p_Q, t_Q, atol=1e-6), "Activations differ more than the allowed tolerance"
# assert torch.allclose(p_K, t_K, atol=1e-6), "Activations differ more than the allowed tolerance"
# assert torch.allclose(p_V, t_V, atol=1e-6), "Activations differ more than the allowed tolerance"

# # qkv bias
# bias_QKV = timm_model.blocks[0].attn.qkv.bias

# b_Q, b_K, b_V = torch.tensor_split(bias_QKV, 3, dim=0)

# bt_Q = einops.rearrange(b_Q, "(i h) -> h i", h=12)
# bt_K = einops.rearrange(b_K, "(i h) -> h i", h=12)
# bt_V = einops.rearrange(b_V, "(i h) -> h i", h=12)

# bp_Q = prisma_model.blocks[0].attn.b_Q
# bp_K = prisma_model.blocks[0].attn.b_K
# bp_V = prisma_model.blocks[0].attn.b_V

# assert torch.allclose(bp_Q, bt_Q, atol=1e-6), "Activations differ more than the allowed tolerance"
# assert torch.allclose(bp_K, bt_K, atol=1e-6), "Activations differ more than the allowed tolerance"
# assert torch.allclose(bp_V, bt_V, atol=1e-6), "Activations differ more than the allowed tolerance"

In [ ]:
def print_matrix_corner(matrix, rows=1, cols=1):
    """
    Prints the top-left corner of a matrix (tensor) up to the specified number of rows and columns.

    Parameters:
    - matrix (torch.Tensor): The matrix (tensor) from which to print the corner.
    - rows (int): The number of rows to include in the printed corner. Default is 5.
    - cols (int): The number of columns to include in the printed corner. Default is 5.
    """
    # Ensure the matrix is a PyTorch tensor
    if not isinstance(matrix, torch.Tensor):
        print("The input is not a PyTorch tensor.")
        return

    # Get the size of the matrix
    num_rows, num_cols = matrix.shape[:2]

    # Adjust rows and cols if the matrix is smaller than specified dimensions
    rows_to_print = min(rows, num_rows)
    cols_to_print = min(cols, num_cols)

    # Slice the matrix to get the top-left corner
    corner = matrix[:rows_to_print, :cols_to_print]

    print(f"Top-left corner ({rows_to_print}x{cols_to_print}):\n{corner}")


**QKV matrix**

In [ ]:
# First layer
activations = []
def hook_fn(module, input, output):
    activations.append(output)

hook_handle = timm_model.blocks[0].attn.qkv.register_forward_hook(hook_fn)

timm_output = timm_model(image)
hook_handle.remove()

print("timm output", activations[0].shape)
# qkv = activations[0].reshape(-1, 197, 3, 12, 64).permute(2, 0, 3, 1, 4)
qkv = activations[0].reshape(-1, 197, 3, 12, 64).permute(2, 0, 3, 1, 4)
q, k, v = qkv.unbind(0)

print("timm shape", qkv.shape)
# print("prisma shape", cache['blocks.0.attn.hook_qkv'].shape)

print("prisma q shape", cache['blocks.0.attn.hook_q'].shape)

# assert torch.allclose(qkv, cache['blocks.0.attn.hook_qkv'], atol=1e-6), "Activations differ more than the allowed tolerance"
assert torch.allclose(q, cache['blocks.0.attn.hook_q'], atol=1e-6), "Activations differ more than the allowed tolerance"
assert torch.allclose(k, cache['blocks.0.attn.hook_k'], atol=1e-6), "Activations differ more than the allowed tolerance"
assert torch.allclose(v, cache['blocks.0.attn.hook_v'], atol=1e-6), "Activations differ more than the allowed tolerance"

**Attention Scores**

In [ ]:
scaled_q = q * 64 ** -0.5
timm_attn_scores = scaled_q @ k.transpose(-2,-1)

print("timm attn scores", timm_attn_scores.shape)
print("prisma attn scores", cache['blocks.0.attn.hook_attn_scores'].shape)

assert torch.allclose(timm_attn_scores, cache['blocks.0.attn.hook_attn_scores'], atol=1e-4), "Activations differ more than the allowed tolerance"

**Attention pattern**

In [ ]:
timm_attn_pattern = timm_attn_scores.softmax(dim=-1) 

assert torch.allclose(timm_attn_pattern, cache['blocks.0.attn.hook_pattern'], atol=1e-4), "Activations differ more than the allowed tolerance"

**Attention Output**

In [ ]:
# First layer
activations = []
def hook_fn(module, input, output):
    activations.append(output)

hook_handle = timm_model.blocks[0].attn.proj.register_forward_hook(hook_fn)
timm_output = timm_model(image)
hook_handle.remove()

print(activations[0].shape)
# print(cache['blocks.0.attn.hook_attn_out'].shape)


assert torch.allclose(cache['blocks.0.attn.hook_result'], activations[0], atol=1e-3), "Activations differ more than the allowed tolerance"


## MLP

In [ ]:

#currently only vit_base_patch16_224 supported (config loading issue)
TOLERANCE = 1e-5

model_name = "vit_base_patch16_224"
batch_size = 5
channels = 3
height = 224
width = 224
device = "cuda"

hooked_model = HookedViT.from_pretrained(model_name)
hooked_model.to(device)
timm_model = timm.create_model(model_name, pretrained=True)
timm_model.to(device)

with torch.random.fork_rng():
    torch.manual_seed(1)
    input_image = torch.rand((batch_size, channels, height, width)).to(device)

assert torch.allclose(hooked_model(input_image), timm_model(input_image), atol=TOLERANCE), "Model output diverges!"


In [ ]:
hooked_model(input_image).shape

In [ ]:
timm_model(input_image).shape